In [9]:
import time
import datetime
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import tushare as ts
ts.set_token('29eaf3bcac23df4c6d025de157ab2d53beead3391fbe6e83b4ebcb6c')
pro = ts.pro_api()

import mpl_finance as mpf
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.pylab import date2num
import matplotlib.gridspec as gridspec
import matplotlib as mpl

mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'SimHei'

In [10]:
import torch
from torch import nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


## create dataset 

In [11]:
from torch.utils.data import Dataset

class MyDatasetZT(Dataset):
    def __init__(self, df, transform=None, target_transform=None):
        self.info_df = df
        self.imgs = df["file_name"].values
        self.labels = df["label"].values
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        filename = self.imgs[index]
        label = self.labels[index]
        img = pd.read_csv(filename)
        img = img.fillna(0)
        img = img.values[:,:11]   # 行，列
        img[-1,2:10] = 0
#         img[-1,11:] = 0
        if self.transform is not None:
            img = self.transform(img)
        return img,label

    def __len__(self):
        return len(self.imgs)

In [13]:
from torch.utils.data import DataLoader

from torchvision import transforms, utils

# Hyper Parameters
EPOCH = 1               # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 256
TIME_STEP = 64          # rnn time step / image height
INPUT_SIZE = 11         # rnn input size / image width
LR = 1e-5             # learning rate
LOAD_MODEL = not True

# test created dataset
info_df = pd.read_csv("./data/deeplearning_data/train_data/"+"info.csv")
dataset_ZT = MyDatasetZT(info_df )
train_loader = DataLoader(dataset_ZT, batch_size=BATCH_SIZE,shuffle=True)

In [15]:
for i, (batch_x, batch_y) in enumerate(train_loader):
    if(i>4):
        break
    print(i, batch_x.size(),batch_y.size())

FileNotFoundError: [Errno 2] File ./data/deeplearning_data/train_data/ZT/600081.SH-20160318.csv does not exist: './data/deeplearning_data/train_data/ZT/600081.SH-20160318.csv'

In [14]:
batch_y

NameError: name 'batch_y' is not defined

In [115]:
# define network
class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=input_size,
            hidden_size=hidden_size,         # rnn hidden unit
            num_layers=num_layers,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
            bidirectional = not True
        )
        self.out = nn.Linear(hidden_size, 2)  # if bidirection the input size should plus 2

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state
        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out

In [117]:
# train
rnn = RNN(input_size = 11, hidden_size = 32, num_layers = 1)
try:
    if LOAD_MODEL:
        print("load model")
        rnn.load_state_dict(torch.load("./rnn.pth"))
except:
    print("no previous cache")
print(rnn)
LR = 1e-5
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR )   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss() 
# loss_func = nn.BCEWithLogitsLoss()                       # the target label is not one-hotted
# loss_func = nn.BCELoss() 

# training and testing
for epoch in range(EPOCH):
    rnn.train()
    train_acc = 0.0
    train_loss = 0.0
    for step, (b_x, b_y) in enumerate(train_loader):        # gives batch data
#        print(b_x.size())
#         b_x = b_x.view(-1, 28, 28)              # reshape x to (batch, time_step, input_size)
#        print(b_x.size())
        b_x = torch.tensor(b_x, dtype=torch.float32)
        output = rnn(b_x)                               # rnn output
#         b_y_onehot = torch.nn.functional.one_hot(b_y, 2) 
#         b_y_onehot = torch.tensor(b_y_onehot, dtype=torch.float32)
        loss = loss_func(output, b_y)                   # cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients
        _, pred_y = torch.max(output, 1)
        
        train_loss += loss*BATCH_SIZE
        train_acc += (pred_y == b_y).float().sum()
            
        if step % 2 == 0:
            accuracy = (pred_y == b_y).float().mean()
            print('step: ', step+1, '| train loss: %.4f' % loss.data, '| train accuracy: %.2f' % accuracy)
    print(f'Train loss: {train_loss/len(train_data): .6f} train acc: {train_acc/len(train_data):.4f}')
        
torch.save(rnn.state_dict(),"./rnn.pth")

RNN(
  (rnn): LSTM(11, 32, batch_first=True)
  (out): Linear(in_features=32, out_features=2, bias=True)
)
step:  1 | train loss: 0.6943 | train accuracy: 0.48
step:  3 | train loss: 0.6963 | train accuracy: 0.48
step:  5 | train loss: 0.6902 | train accuracy: 0.54
step:  7 | train loss: 0.6927 | train accuracy: 0.52
step:  9 | train loss: 0.6922 | train accuracy: 0.51
step:  11 | train loss: 0.6921 | train accuracy: 0.51
step:  13 | train loss: 0.6906 | train accuracy: 0.55
step:  15 | train loss: 0.6900 | train accuracy: 0.54
step:  17 | train loss: 0.6923 | train accuracy: 0.49
step:  19 | train loss: 0.6931 | train accuracy: 0.49
step:  21 | train loss: 0.6904 | train accuracy: 0.55
step:  23 | train loss: 0.6929 | train accuracy: 0.53
step:  25 | train loss: 0.6914 | train accuracy: 0.54
step:  27 | train loss: 0.6947 | train accuracy: 0.46
step:  29 | train loss: 0.6907 | train accuracy: 0.55
step:  31 | train loss: 0.6931 | train accuracy: 0.48
step:  33 | train loss: 0.6910 | tr

KeyboardInterrupt: 

In [ ]:
# evaluate
'''
    rnn.eval()
    test_loss = 0.0
    test_acc = 0.0
    for i, (b_x, b_y) in enumerate(test_loader):
        b_x = b_x.view(-1, 28, 28)              # reshape x to (batch, time_step, input_size)
#        assert c == 1, 'channel must be 1'
#        img = img.squeeze(1)
#        print(b_x.size())
        if use_gpu:
            b_x = b_x.cuda()
            b_y = b_y.cuda()
        output = rnn(b_x)                               # rnn output
        loss = loss_func(output, b_y) 
        
        _, pred_y = torch.max(output, 1)
        accuracy = (pred_y == b_y).float().sum()
        test_loss += loss*BATCH_SIZE
        test_acc += accuracy
        
    print(f'Test loss: {test_loss/len(test_data): .6f} test acc: {test_acc/len(test_data):.4f}')
    print()'''